In [27]:
import numpy as np
import pandas as pd # Pandas is an easy-to-use data structures and data analysis tools
pd.set_option('display.max_columns', None) # To display all columns

In [28]:
df = pd.read_csv("dataset.csv")

In [29]:
df.head()

,Bug ID,Product,Component,Assignee,Status,Resolution,Summary,Changed,Assignee Real Name,Classification,Flags,Hardware,Keywords,Number of Comments,Opened,OS,Priority,QA Contact,QA Contact Real Name,Reporter,Reporter Real Name,Severity,Summary.1,Tags,Target Milestone,URL,Version,Votes,Whiteboard,Alias
0,3638,JDT,UI,aeschli,VERIFIED,FIXED,Package Viewer: order resource folders before ...,2002-01-17 07:28:04,Martin Aeschlimann,Eclipse,NaN,All,NaN,3,2001-10-10 22:58:24,Windows NT,P1,NaN,NaN,aeschli,Martin Aeschlimann,major,Package Viewer: order resource folders before ...,NaN,---,NaN,2.0,0,NaN,NaN
1,3854,JDT,UI,aeschli,VERIFIED,FIXED,Wrong execution's classpath. (1GEY0W0),2002-01-18 04:02:11,Martin Aeschlimann,Eclipse,NaN,All,NaN,5,2001-10-10 23:01:58,Windows NT,P1,NaN,NaN,david_audel,David Audel,normal,Wrong execution's classpath. (1GEY0W0),NaN,---,NaN,2.0,0,NaN,NaN
2,4188,JDT,UI,aeschli,VERIFIED,FIXED,type hierachy - typo (1GJW2XJ),2002-01-28 03:12:37,Martin Aeschlimann,Eclipse,NaN,All,NaN,3,2001-10-10 23:07:31,Windows 2000,P1,NaN,NaN,erich_gamma,Erich Gamma,normal,type hierachy - typo (1GJW2XJ),NaN,---,NaN,2.0,0,NaN,NaN
3,5115,JDT,Debug,aeschli,VERIFIED,FIXED,Workspace source locator fails with mulitple p...,2001-11-13 10:11:09,Martin Aeschlimann,Eclipse,NaN,PC,NaN,10,2001-10-19 13:41:25,Windows 2000,P1,NaN,NaN,darin.eclipse,Darin Wright,normal,Workspace source locator fails with mulitple p...,NaN,---,NaN,2.0,0,NaN,NaN
4,5820,JDT,UI,aeschli,VERIFIED,FIXED,Close all editors brings up hierarchy of object,2001-11-20 16:22:35,Martin Aeschlimann,Eclipse,NaN,PC,NaN,5,2001-11-12 18:18:05,Windows 2000,P1,NaN,NaN,jed.anderson,Jed Anderson,normal,Close all editors brings up hierarchy of object,NaN,---,NaN,2.0,0,NaN,NaN


In [30]:
df['Component'].value_counts()

UI                                          4019
Debug                                       2928
Text                                         875
Ant                                          729
Core                                         490
SWT                                          145
User Assistance                              116
Releng                                       104
Doc                                           90
Compare                                       85
Team                                          85
IDE                                           71
Search                                        68
API Tools                                     60
CVS                                           52
Update  (deprecated - use RT>Equinox>p2)      31
Resources                                     18
Runtime                                       10
Build                                         10
APT                                            9
PMC                 

In [31]:
df['Component'] = df['Component'].astype('str')

In [32]:
with open("dataset.csv", 'r', encoding="utf8") as f:
    lines = f.readlines()
    components = []
    assignees = []
    summaries = []
    labels = []
    for d in lines:
        bugid,product,component,assignee,status,resolution,summary,_ = d.split(',')[:8]
        if(component == '"UI"'):
            labels.append(1)
        elif(component == '"Debug"'):
            labels.append(0)
        elif(component == '"Text"'):
            labels.append(2)
        elif(component == '"Ant"'):
            labels.append(3)
        elif(component == '"Core"'):
            labels.append(4)
        else:
            labels.append(5)
        components.append(component)
        assignees.append(assignee)
        summaries.append(summary)
    len(labels)

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z][a-zA-Z]\w+\b")
data = []
for i in summaries:
    data.append(i.strip())

In [34]:
from sklearn.feature_extraction.text import TfidfTransformer
X_train_counts = count_vect.fit_transform(data)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [35]:
X_train_tfidf = X_train_tfidf[:int(X_train_tfidf.shape[0] * 0.7)]

In [36]:
from sklearn.naive_bayes import MultinomialNB
labels = labels[:int(len(labels) * 0.7)]
clf = MultinomialNB().fit(X_train_tfidf, labels)

In [37]:
labels_test = labels[int(len(labels) * 0.7):]
X_test_tfidf = X_train_tfidf[int(X_train_tfidf.shape[0] * 0.7):]

In [38]:
predicted = clf.predict(X_test_tfidf)

In [39]:
print("\nAccuracy Training: " + str(np.mean(predicted == labels_test)))


Accuracy Training: 0.662380952381
